In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("sample_data.csv")

# probably change key to machine_key and add a factory_key
df["key"] = df["PID"].astype('str')

In [3]:
PIDs = list(df["key"].unique())
PIDs.sort()

In [4]:
df.head(3)

,footfall,atemp,selfLR,ClinLR,DoleLR,PID,outpressure,inpressure,temp,fail,key
0,0,7,7,1,6,6,36,3,1,pass,6
1,190,1,3,3,5,1,20,4,1,fail,1
2,31,7,2,2,6,1,24,6,1,fail,1


In [5]:
def split(df: pd.DataFrame):
  """
  splits data into a bunch of groups to simulate live data... just a quick hack
  """
  temp = []
  num_chunks = int(len(df)/11)
  for i in range(num_chunks):
    temp.append(df[i:i+10])

  return temp

In [11]:
class Machine:
  def __init__(self, machine_key: str):
    self.machine_key = machine_key
    self.samples_to_save = 5
    self.historic = dict({})
    self.columns = None

  def update_historic(self, data: pd.Series):
    key = data["key"]
    if key in self.historic:
      self.historic.get(key).append(data.values)
      if len(self.historic.get(key)) > self.samples_to_save:
        self.historic[key] = self.historic.get(key)[-self.samples_to_save:]
    else:
      self.historic[key] = [data.values]
      self.columns = data.index

In [12]:
class Factory:
  def __init__(self):
    self.machines = dict({})

  def add_machine(self, machine_key: str, machine: Machine):
    self.machines[machine_key] = Machine(machine_key)

  def update_machine(self, machine_key: str, data_stream: pd.DataFrame):
    if machine_key in self.machines:
      self.machines[machine_key].update_historic(data_stream)
    else:
      machine = Machine(machine_key)
      machine.update_historic(data_stream)
      self.add_machine(machine_key, machine)

In [13]:
def simulate_factory_running(simulated_data):
  factory = Factory()
  for data_stream in simulated_data:
    for i in range(len(data_stream)):
      machine_key = data_stream.iloc[i]["key"]
      factory.update_machine(machine_key, data_stream.iloc[i])

  return factory

In [14]:
def print_factory_machine_data(factory):
  for machine_key in factory.machines.keys():
    print(factory.machines[machine_key].historic)

In [15]:
simulated_data = split(df)
factory = simulate_factory_running(simulated_data)

print_factory_machine_data(factory)

{'6': [array(['15', '7', '6', '2', 6, 6, 59, 1, '5', 'pass', '6'], dtype=object), array(['15', '7', '6', '2', 6, 6, 59, 1, '5', 'pass', '6'], dtype=object), array(['31', '2', '3', '4', 6, 6, 20, 4, '6', 'pass', '6'], dtype=object), array(['31', '2', '3', '4', 6, 6, 20, 4, '6', 'pass', '6'], dtype=object), array(['31', '2', '3', '4', 6, 6, 20, 4, '6', 'pass', '6'], dtype=object)]}
{'1': [array(['59', '5', '3', '3', 5, 1, 73, 5, '5', 'fail', '1'], dtype=object), array(['59', '5', '3', '3', 5, 1, 73, 5, '5', 'fail', '1'], dtype=object), array(['59', '5', '3', '3', 5, 1, 73, 5, '5', 'fail', '1'], dtype=object), array(['59', '5', '3', '3', 5, 1, 73, 5, '5', 'fail', '1'], dtype=object), array(['1600', '1', '3', '3', 6, 1, 24, 6, '6', 'fail', '1'], dtype=object)]}
{'0': [array(['31', '7', '2', '2', 7, 0, 22, 4, '6', 'fail', '0'], dtype=object), array(['35', '7', '3', '2', 5, 0, 71, 2, '5', 'fail', '0'], dtype=object), array(['170', '7', '4', '3', 2, 0, 48, 2, '6', 'fail', '0'], dtype=object),